In [1]:
import os
print("=================================切换目录=====================================")
print(os.getcwd()) # 打印当前工作目录
os.chdir('/home/shichaoyu/perl5/IRpro2/dataset/')
print(os.getcwd())
print(os.listdir())

=================================切换目录=====================================
/home/shichaoyu/perl5/IRpro2
/home/shichaoyu/perl5/IRpro2/dataset
['train', 'cat_feature_index.3c4c9673-11b0f6ac-cb85fb96-91833486.tmp', 'catboost_info', 'cat_feature_index.40b30d8b-f30e5088-cc5f7d50-21dddf23.tmp', 'out', 'cna_test_data', 'cna_data', 'pkl2', 'cat_feature_index.45ed2826-847e7041-c540e66c-339d9b99.tmp', 'cat_feature_index.3be0c692-8a8fe0c8-bb8954c3-56fb0ff.tmp', 'feat']


In [ ]:
## 1. to_pickle: 将数据存为表格格式
import json
import os
import numpy as np
import pandas as pd
from collections import OrderedDict

pkl_path = './pkl2'
if not os.path.exists(pkl_path):
    os.mkdir(pkl_path)

    
# 测试集实例提交文件    cna_test_data
with open("cna_test_data/test_example_evaluation_continuous.json")as file:
    valid_example=json.load(file, object_pairs_hook=OrderedDict)
# 待分类文件
with open("cna_test_data/cna_test_unass_competition.json")as file:
    cna_valid_unass=json.load(file, object_pairs_hook=OrderedDict)
#  分类文件元数据
with open("cna_test_data/cna_test_pub.json")as file:
    cna_valid_pub=json.load(file, object_pairs_hook=OrderedDict)
    
# #示例提交文件
# with open('cna_data/valid_example_evaluation_continuous.json') as file:
#     valid_example = json.load(file, object_pairs_hook=OrderedDict)
# # 论文元信息
# with open('cna_data/cna_valid_pub.json') as file:
#     cna_valid_pub = json.load(file, object_pairs_hook=OrderedDict)
    
# #  待分配的论文列表
# with open('cna_data/cna_valid_unass_competition.json') as file:
#     cna_valid_unass = json.load(file, object_pairs_hook=OrderedDict)

# 作者档案    已有用户档案
with open('cna_data/whole_author_profile.json') as file:
    whole_author_profile = json.load(file, object_pairs_hook=OrderedDict)
#     二级字典，key值为作者id，value分为俩个域: ‘name’域代表作者名，’papers’域代表作者的所拥有的论文(作者的profile)
#    , 测试集与验证集使用同一个已有的作者档案；

# 涉及的论文元信息
with open('cna_data/whole_author_profile_pub.json') as file:
    whole_author_profile_pub = json.load(file, object_pairs_hook=OrderedDict)

# 训练集   二级字典   作者姓名，作者ID，论文ID列表   一个作者对应许多ID，一个ID对应多篇论文
# 论文元数据
with open('train/train_pub.json') as file:
    train_pub = json.load(file, object_pairs_hook=OrderedDict)
with open('train/train_author.json') as file:
    train_author = json.load(file, object_pairs_hook=OrderedDict)

#  将数据成对保存，使用pandas处理，并使用pickle保存
### train_author

# 1. author_name, author_ids    作者姓名和ID  
train_author_names = list(train_author.keys())
train_author_ids = [list(v.keys()) for v in train_author.values()]
train_author_name_ids = pd.DataFrame(list(zip(train_author_names, train_author_ids)), columns=['author_name', 'author_ids'])

# train_author_name_ids.head()

train_author_name_ids.to_pickle('./pkl2/train_author_name_ids.pkl')

# 2. author_id, paper_ids   作者ID和论文ID  
train_author_paper_ids = pd.DataFrame([(k2, v2) for v1 in train_author.values() for k2, v2 in v1.items()], columns=['author_id', 'paper_ids'])

# train_author_paper_ids.head()

train_author_paper_ids.to_pickle('./pkl2/train_author_paper_ids.pkl')

### train_pub

# paper_id, author_names&orgs, title, venue, year, keywords, abstract
train_pub_info = pd.DataFrame.from_dict(train_pub, orient='index').reset_index(drop=True).rename({'id':'paper_id'}, axis=1)

# train_pub_info.head()

train_pub_info.to_pickle('./pkl2/train_pub_info.pkl')

### whole_author_profile

# 其键（key）是论文ID，其值是相应的论文信息。 

whole_author_name_paper_ids = pd.DataFrame.from_dict(whole_author_profile, orient='index').reset_index()
whole_author_name_paper_ids.columns = ['author_id', 'author_name', 'paper_ids']

# whole_author_name_paper_ids.head()

whole_author_name_paper_ids.to_pickle('./pkl2/whole_author_name_paper_ids.pkl')

### whole_author_profile_pub

# paper_id, author_names&orgs, title, venue, year, keywords, abstract
whole_pub_info = pd.DataFrame.from_dict(whole_author_profile_pub, orient='index').reset_index(drop=True).rename({'id':'paper_id'}, axis=1)

# whole_pub_info.head()

whole_pub_info.to_pickle('./pkl2/whole_pub_info.pkl')

### cna_valid_unass

# 论文列表，代表待分配的论文list，列表中的元素为论文id + ‘-’ + 需要分配的作者index(从0开始)；
# 参赛者需要将该文件中的每篇论文的待分配作者对应分配到已有作者档案中(whole_author_profile.json).

cna_valid_unass = pd.DataFrame(cna_valid_unass, columns=['cna_valid_unass'])

cna_valid_unass['cna_valid_unass'] = cna_valid_unass['cna_valid_unass'].apply(lambda x: x.split('-'))

cna_valid_unass['paper_id'] = cna_valid_unass['cna_valid_unass'].apply(lambda x: x[0])
cna_valid_unass['author_idx'] = cna_valid_unass['cna_valid_unass'].apply(lambda x: x[1])

del cna_valid_unass['cna_valid_unass']

# cna_valid_unass.head()

cna_valid_unass.to_pickle('./pkl2/cna_valid_unass.pkl')

### cna_valid_pub

# paper_id, author_names&orgs, title, venue, year, keywords, abstract
valid_pub_info = pd.DataFrame.from_dict(cna_valid_pub, orient='index').reset_index(drop=True).rename({'id':'paper_id'}, axis=1)

# valid_pub_info.head()

valid_pub_info.to_pickle('./pkl2/valid_pub_info.pkl')

### pub info

pub_info = pd.concat([whole_pub_info, train_pub_info, valid_pub_info]).drop_duplicates(subset='paper_id', keep='first')

pub_info['orgs'] = pub_info['authors'].apply(lambda x: [ao['org'] for ao in x if 'org' in ao])
pub_info['authors'] = pub_info['authors'].apply(lambda x: [ao['name'] for ao in x if 'name' in ao])

pub_info['year'] = pub_info['year'].fillna(0).replace('', 0).astype(int)

pub_info['abstract'] = pub_info['abstract'].fillna(' ').replace('', ' ')

# pub_info.head()

pub_info.to_pickle('./pkl2/pub_info.pkl')

### author_pub_detail

author_pub_ids = whole_author_name_paper_ids[['author_id','paper_ids']].merge(train_author_paper_ids, 'left', 'author_id')

author_pub_ids['paper_ids_x_len'] = author_pub_ids['paper_ids_x'].apply(len)
author_pub_ids['paper_ids_y_len'] = author_pub_ids['paper_ids_y'].apply(lambda x: 0 if type(x) == float else len(x))

author_pub_ids['paper_ids'] = author_pub_ids.apply(lambda row: list(set(row['paper_ids_x']) | (set() if type(row['paper_ids_y']) == float else set(row['paper_ids_y']))), axis=1)

author_pub_ids['paper_ids_len'] = author_pub_ids['paper_ids'].apply(len)

author_pub_ids.drop(columns=['paper_ids_x', 'paper_ids_y', 'paper_ids_x_len', 'paper_ids_y_len'], inplace=True)

# author_pub_ids.head()

author_pub_ids['paper_ids_len'].describe()

pub_info = pub_info.set_index('paper_id')

# pub_info.head()

# author_id paper_ids paper_ids_len abstracts keywords titles venues years authors orgs

from tqdm import tqdm_notebook

author_pub_ids_ = author_pub_ids[['author_id', 'paper_ids']].values
pub_col = ['abstract', 'keywords', 'title', 'venue', 'year', 'authors', 'orgs']
for pc in pub_col:
    print(pc)
    dat = []
    for author_id, paper_ids in tqdm_notebook(author_pub_ids_):
        d = []
        for pid in paper_ids:
            d.append(pub_info.loc[pid, pc])
        dat.append(d)
    author_pub_ids[pc] = dat

# author_pub_ids.head()

len(author_pub_ids[author_pub_ids['author_id'] == 'sCKCrny5']['abstract'].values[0])

author_pub_ids['year'].apply(len).describe()

author_pub_ids.to_pickle('./pkl2/author_pub_detail.pkl')

In [3]:
## 2. gen_valid 生成测试样本
import pandas as pd
import numpy as np
import os
import sys
from collections import OrderedDict
import json

### valid data

valid_pub_info = pd.read_pickle('./pkl2/valid_pub_info.pkl')      #pub信息
cna_valid_unass = pd.read_pickle('./pkl2/cna_valid_unass.pkl')    # 验证集信息

valid_data = cna_valid_unass.merge(valid_pub_info, 'left', 'paper_id')

valid_data['author_idx'] = valid_data['author_idx'].astype(int)
print(valid_data['author_idx'].max())

valid_data['author_name'] = valid_data.apply(lambda row: row['authors'][row['author_idx']]['name'], axis=1)

valid_data['author_org'] = valid_data.apply(lambda row: row['authors'][row['author_idx']].get('org'), axis=1)

valid_data = valid_data[['paper_id', 'author_name', 'author_org']]
# 将作者姓名转换为统一格式 ，名中间不保留任何字符，姓氏和名字之间保留一个下划线字符
def convert(name):
    name = name.lower()
    name = name.replace('. ', '_').replace('.', '_').replace(' ', '_').replace('-', '')
    if name in ['yang_jie', 'jie_yang_0002', 'jie\xa0yang', 'jie_yang_0008']:
        name = 'jie_yang'
    if name in ['liu_bing']:
        name = 'bing_liu'
    return name

valid_data['author_name'] = valid_data['author_name'].apply(convert)

# valid_data.head()

valid_data['author_name'].nunique()

valid_data[valid_data['author_org'] == 'South China University of Technology']



whole_author_name_paper_ids = pd.read_pickle('./pkl2/whole_author_name_paper_ids.pkl')

whole_author_name_paper_ids.head()

valid_data_ = valid_data.merge(whole_author_name_paper_ids[['author_name', 'author_id']], 'left', 'author_name')

valid_data_.isnull().sum() / len(valid_data_)

# valid_data_.head()

valid_data_.to_pickle('./pkl2/valid_data.pkl')

2841


In [4]:
## 3. author_name_match 作者名匹配
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import sys
import pypinyin

# !pip install pypinyin
# 检测中文，Unicode编码 
def check_chs(c):
    return '\u4e00' <= c <= '\u9fa5'

def to_pinyin(word):
    s = ''
    for i in pypinyin.pinyin(word, style=pypinyin.NORMAL):
        s += ''.join(i)
    return s

##   全部信息
whole_pub_info = pd.read_pickle('./pkl2/whole_pub_info.pkl')
whole_author_name_paper_ids = pd.read_pickle('./pkl2/whole_author_name_paper_ids.pkl')
##   训练集信息
train_pub_info = pd.read_pickle('./pkl2/train_pub_info.pkl')
train_author_name_ids = pd.read_pickle('./pkl2/train_author_name_ids.pkl')
train_author_paper_ids = pd.read_pickle('./pkl2/train_author_paper_ids.pkl')
##   验证集信息
valid_pub_info = pd.read_pickle('./pkl2/valid_pub_info.pkl')
cna_valid_unass = pd.read_pickle('./pkl2/cna_valid_unass.pkl')

# 将数据索引对齐，进行合并
pub_info = pd.concat([whole_pub_info, train_pub_info, valid_pub_info])
print(pub_info.shape)
# 去除重复项
pub_info = pub_info.drop_duplicates(subset='paper_id', keep='first')
print(pub_info.shape)

whole_author_name_paper_ids.head()

paper_authors = {}
for author_name, paper_ids in whole_author_name_paper_ids[['author_name', 'paper_ids']].values:
    for pid in paper_ids:
        if not pid in paper_authors:
            paper_authors[pid] = [author_name]
        else:
            paper_authors[pid].append(author_name)

paper_authors_df = pd.DataFrame([(k, v) for k,v in paper_authors.items()], columns=['paper_id', 'author_ids'])

pub_info['author_names'] = pub_info['authors'].apply(lambda x: [ao['name'] for ao in x])

pub_info = pub_info.merge(paper_authors_df, 'left', 'paper_id')

pub_info.head()

from collections import defaultdict
def score(n1, n2):
    n1 = ''.join(filter(str.isalpha, n1.lower()))
    if check_chs(n1):
#         print(n1)
        n1 = to_pinyin(n1)
#         print(n1)
    n2 = ''.join(filter(str.isalpha, n2.lower()))
    counter = defaultdict(int)
    score = 0
    for c in n1:
        counter[c] += 1
    for c in n2:
        if (c in counter) and (counter[c] > 0):
            counter[c] -= 1
        else:
            score += 1
    score += np.sum(list(counter.values()))
    return score
    
score('hello world', 'world hello')

from tqdm import tqdm_notebook
author_name_map = {}
for author_names, author_ids in tqdm_notebook(pub_info[['author_names', 'author_ids']].values):
    if type(author_ids) == float:
        continue
    for aid in author_ids:
        dis = []
        for an in author_names:
            dis.append(score(an, aid))
        cor = author_names[np.argmin(dis)]
        author_name_map[cor] = aid

for k in author_name_map.keys():
    if check_chs(k):
        print(k)

import pickle
with open('./pkl2/author_name_map.pkl', 'wb') as file:
    pickle.dump(author_name_map, file)
print(82*'-')
author_name_map

len(author_name_map)

(475195, 7)
(271719, 7)



王慧
刘晶 liu jing
刘建国 liu jianguo
胡斌
----------------------------------------------------------------------------------


2164

In [5]:
## 4. author_org_match 作者单位匹配
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import sys

import pickle

##    全部信息
whole_pub_info = pd.read_pickle('./pkl2/whole_pub_info.pkl')
whole_author_name_paper_ids = pd.read_pickle('./pkl2/whole_author_name_paper_ids.pkl')
##   训练集
train_pub_info = pd.read_pickle('./pkl2/train_pub_info.pkl')
train_author_name_ids = pd.read_pickle('./pkl2/train_author_name_ids.pkl')
train_author_paper_ids = pd.read_pickle('./pkl2/train_author_paper_ids.pkl')
##   验证集
valid_pub_info = pd.read_pickle('./pkl2/valid_pub_info.pkl')
valid_data = pd.read_pickle('./pkl2/valid_data.pkl')
#   合并
pub_info = pd.concat([whole_pub_info, train_pub_info, valid_pub_info])
print(pub_info.shape)
pub_info = pub_info.drop_duplicates(subset='paper_id', keep='first')
print(pub_info.shape)

pub_info.head(2)

pub_info = pub_info.set_index('paper_id')
pub_info.head(2)

pub_info.loc['0009qJgC', 'authors']

with open('./pkl2/author_name_map.pkl', 'rb') as file:
    author_name_map = pickle.load(file)

whole_author_name_paper_ids.head(2)

print(train_author_paper_ids.head(2))

print(valid_data.head(2))

# dict: {author_name:{paper_id:org}}

train_author_name_ids = pd.read_pickle('./pkl2/train_author_name_ids.pkl')

train_author_name_ids_ext = []
for author_name, author_ids in train_author_name_ids[['author_name', 'author_ids']].values:
     for aid in author_ids:
            train_author_name_ids_ext.append([author_name, aid])
            
train_author_name_ids_ext = pd.DataFrame(train_author_name_ids_ext, columns=['author_name', 'author_id'])
train_author_name_ids_ext.head()

train_author_name_paper_ids = train_author_paper_ids.merge(train_author_name_ids_ext, 'left', 'author_id')
print(train_author_name_paper_ids.head(2))

author_name_paper_ids = pd.concat([train_author_name_paper_ids, whole_author_name_paper_ids])
print(author_name_paper_ids.head(2))

# dict: {author_name:{paper_id:org}}
from tqdm import tqdm_notebook
author_org_map = {}
for author_name, paper_ids in tqdm_notebook(author_name_paper_ids[['author_name', 'paper_ids']].values):
    if not author_name in author_org_map:
        author_org_map[author_name] = {}
    for pid in paper_ids:
        org = np.nan
        author_orgs = pub_info.loc[pid, 'authors']
        for ao in author_orgs:
            if not ao['name'] in author_name_map:
                continue
            if author_name_map[ao['name']] == author_name:
                org = ao.get('org')
        author_org_map[author_name][pid] = org

author_org_map['li_guo']

author_org_map['li_guo']['UG32p2zs']

with open('./pkl2/author_org_map.pkl', 'wb') as file:
    pickle.dump(author_org_map, file)
    

(475195, 7)
(271719, 7)
  author_id                                          paper_ids
0  EShnTfSe                                         [P9a1gcvg]
1  sCKCrny5  [Rg5fAeTd, lJPsGNBE, er5gTz90, UG32p2zs, FTweB...
   paper_id author_name                                         author_org  \
0  5Ix5q3pI   xiong_hui                                               None   
1  8X1mND8h  zhang_tong  The College of Life Science,Henan University,K...   

  author_id  
0       NaN  
1       NaN  
  author_id                                          paper_ids author_name
0  EShnTfSe                                         [P9a1gcvg]      li_guo
1  sCKCrny5  [Rg5fAeTd, lJPsGNBE, er5gTz90, UG32p2zs, FTweB...      li_guo
  author_id author_name                                          paper_ids
0  EShnTfSe      li_guo                                         [P9a1gcvg]
1  sCKCrny5      li_guo  [Rg5fAeTd, lJPsGNBE, er5gTz90, UG32p2zs, FTweB...


In [6]:
%%time
## 5 gen_train 生成训练样本
import pandas as pd
import numpy as np
import os
import sys
from collections import OrderedDict
import json
import random
random.seed(42)
np.random.seed(42)

import pickle
with open('./pkl2/author_name_map.pkl', 'rb') as file:
    author_name_map = pickle.load(file)
with open('./pkl2/author_org_map.pkl', 'rb') as file:
    author_org_map = pickle.load(file)

whole_author_name_paper_ids = pd.read_pickle('./pkl2/whole_author_name_paper_ids.pkl')
train_author_name_ids = pd.read_pickle('./pkl2/train_author_name_ids.pkl')
valid_data = pd.read_pickle('./pkl2/valid_data.pkl')

print('valid: ', len(set(valid_data['author_name'])))
print('train: ', len(set(train_author_name_ids['author_name'])))
print('whole: ', len(set(whole_author_name_paper_ids['author_name'])))
print('-'*60)
print('valid & train: ', len(set(train_author_name_ids['author_name']) & set(valid_data['author_name'])))
print('whole & valid: ', len(set(whole_author_name_paper_ids['author_name']) & set(valid_data['author_name'])))
print('whole & train: ', len(set(whole_author_name_paper_ids['author_name']) & set(train_author_name_ids['author_name'])))

train_author_paper_ids = pd.read_pickle('./pkl2/train_author_paper_ids.pkl')

# whole_author_name_paper_ids

# train_author_name_ids

# train_author_paper_ids

train_author_name_ids['author_num'] = train_author_name_ids['author_ids'].apply(len)
#  数量大于2 
train_author_name_ids = train_author_name_ids[train_author_name_ids['author_num'] >= 2]

print(train_author_name_ids['author_num'].min())

train_author_name_ids

train_author_name_ids_ext = []
for author_name, author_ids in train_author_name_ids[['author_name', 'author_ids']].values:
     for aid in author_ids:
            train_author_name_ids_ext.append([author_name, aid])
            
train_author_name_ids_ext = pd.DataFrame(train_author_name_ids_ext, columns=['author_name', 'author_id'])
train_author_name_ids_ext.head()

train_author_paper_ids_ext = []
for author_id, paper_ids in train_author_paper_ids[['author_id', 'paper_ids']].values:
     for pid in paper_ids:
            train_author_paper_ids_ext.append([author_id, pid])
train_author_paper_ids_ext = pd.DataFrame(train_author_paper_ids_ext, columns=['author_id', 'paper_id'])

# train_pub_info = pd.read_pickle('./pkl/train_pub_info.pkl')[['paper_id', 'authors']]

# train_author_paper_ids_ext = train_author_paper_ids_ext.merge(train_pub_info, 'left', 'paper_id')

train_author_paper_ids_ext = train_author_paper_ids_ext.merge(train_author_name_ids_ext, 'left', 'author_id')

train_author_paper_ids_ext.head()


train_author_paper_ids_ext['author_org'] = train_author_paper_ids_ext.apply(lambda row: author_org_map[row['author_name']][row['paper_id']], axis=1)

train_author_paper_ids_ext['author_org'].nunique() / len(train_author_paper_ids_ext)

train_author_paper_ids_ext.head()

train_author_name_ids_ext.head()

train_author_name_ids_ext2 = train_author_name_ids_ext.merge(train_author_name_ids[['author_name', 'author_ids']], 'left', 'author_name')

train_author_name_ids_ext2.head()

# sample 采样的方式！！！
n = 15
train_author_name_ids_ext2['author_ids_sample'] = train_author_name_ids_ext2['author_ids'].apply(lambda x: np.random.permutation(x)[:n])

train_author_name_ids_ext2['author_ids_sample'] = train_author_name_ids_ext2.apply(lambda row: {row['author_id']} | set(row['author_ids_sample']), axis=1)
train_author_name_ids_ext2.head()

train_author_ids_ext2_sample = []
for author_id, author_ids_sample in train_author_name_ids_ext2[['author_id', 'author_ids_sample']].values:
     for aid in author_ids_sample:
            train_author_ids_ext2_sample.append([author_id, aid])
            
train_author_ids_ext2_sample = pd.DataFrame(train_author_ids_ext2_sample, columns=['author_id', 'author_id_sample'])
train_author_ids_ext2_sample.head()

train_data = train_author_paper_ids_ext.merge(train_author_ids_ext2_sample, 'left', 'author_id')

train_data['label'] = (train_data['author_id'] == train_data['author_id_sample']).astype(int)

train_data.drop(columns=['author_id'], inplace=True)

train_data.columns = ['paper_id', 'author_name', 'author_org', 'author_id', 'label']

train_data['label'].value_counts()

train_data.head()

train_data.to_pickle('./pkl2/train_data.pkl')

valid:  186
train:  221
whole:  320
------------------------------------------------------------
valid & train:  1
whole & valid:  65
whole & train:  220
2
CPU times: user 17 s, sys: 1.33 s, total: 18.3 s
Wall time: 14.4 s


In [7]:
%%time
## 6 gen_feat_v1 构造特征
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import gc
import time
import os

feat_dir = './feat/'
if not os.path.exists(feat_dir):
    os.mkdir(feat_dir)

train_data = pd.read_pickle('./pkl2/train_data.pkl')
test_data = pd.read_pickle('./pkl2/valid_data.pkl')

pub_info = pd.read_pickle('./pkl2/pub_info.pkl')
author_pub_detail = pd.read_pickle('./pkl2/author_pub_detail.pkl')

train_data.head(3)

test_data.head(3)

print(train_data.shape)
print(test_data.shape)

data = pd.concat([train_data, test_data]).reset_index(drop=True)

data.head()

pub_info.columns = ['abstract_a', 'authors_a', 'keywords_a', 'paper_id', 'title_a', 'venue_a', 'year_a', 'orgs_a']
pub_info.head()

author_pub_detail.columns = ['author_id', 'paper_ids', 'paper_ids_len', 'abstract_b', 'keywords_b', 'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b']
author_pub_detail.head()

data = data.merge(pub_info, 'left', 'paper_id').merge(author_pub_detail, 'left', 'author_id')

print(data.shape)

data.head(3)

### delete paper_id in pos sample

def pidx(p, ps):
    ans = np.nan
    for i, p2 in enumerate(ps):
        if p == p2:
            ans = i
            break
    return ans


data['idx'] = data.apply(lambda row: pidx(row['paper_id'], row['paper_ids']) if row['label'] == 1 else np.nan, axis=1)

from tqdm import tqdm_notebook

cols = ['abstract_b', 'keywords_b', 'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b']
for i in tqdm_notebook(range(len(data))):
    if pd.isna(data.loc[i, 'idx']):
        continue
    for c in cols:
        v = list(data.loc[i, c])
        del v[data.loc[i, 'idx'].astype(int)]
        data.set_value(i, c, v)
#         data.loc[i, c] = frozenset(v)

data.to_pickle('./pkl2/data.pkl')

print(data.columns)
print(82*'=')


### year
import pandas as pd
import math
import numpy as np
# print(np.nan)
data = pd.read_pickle('./pkl2/data.pkl')  # 1388713
# print(data.head(1))
# print(data['year_b'].shape[0])
# print(len(data['year_b'][1]))
def myfunc(x):
    if isinstance(x,float):
        x=[0]
        return x
    else:
        if len(x)==0:
            x=[0]
        return x
# for i in range(2185631,2185640):
# #     if math.isnan(data['year_b'][i]):
# #         print(i)
#     if isinstance(data['year_b'][i],float):
#         print(i)
#     print(type(data['year_b'][i]))
#     print(data['year_b'][i])
# print(data['year_b'][2185634])
# for i in range(1388713,data['year_b'].shape[0]):
#     print(i)
#     if len(data['year_b'][i])==0:
#         data['year_b'][i]=[0]
# print(data['year_b'].head(2))
data['year_b'] = data['year_b'].apply(lambda x: myfunc(x))

data['year_b_min'] = data['year_b'].apply(np.min)
data['year_b_max'] = data['year_b'].apply(np.max)
data['year_b_mean'] = data['year_b'].apply(np.mean)
data['year_b_std'] = data['year_b'].apply(np.std)

data['year_b_mm2'] = (data['year_b_min'] + data['year_b_max']) / 2
data['year_b_ai2'] = (data['year_b_max'] - data['year_b_min']) 

for c in ['year_b_min', 'year_b_max', 'year_b_mean', 'year_b_mm2']:
    data[c + '-year_a'] = data[c] - data['year_a']

data['year_inside_range'] = ((data['year_b_min-year_a'] <= 0) & (data['year_b_max-year_a'] >= 0)).astype(int)

cols = ['year_a', 'year_b_min', 'year_b_max', 'year_b_mean', 'year_b_std', 'year_b_mm2',
       'year_b_min-year_a', 'year_b_max-year_a', 'year_b_mean-year_a',
       'year_b_mm2-year_a', 'year_inside_range','year_b_ai2']

data[cols].to_pickle('./feat/time_feat_a.pkl')


### 'authors', 'orgs'

tmp = data[['author_org', 'authors_a', 'orgs_a', 'authors_b', 'orgs_b']]
tmp['authors_a']=tmp['authors_a'].apply(lambda x: [] if not isinstance(x,list) else x)
tmp['orgs_a']=tmp['orgs_a'].apply(lambda x:[] if not isinstance(x,list) else x)
tmp['authors_b']=tmp['authors_b'].apply(lambda x:[] if not isinstance(x,list) else x)
tmp['orgs_b']=tmp['orgs_b'].apply(lambda x:[] if not isinstance(x,list) else x)
def func(a, b):
    cnt = 0
    for x in b:
        for y in x:
            if a == y:
                cnt += 1
    return cnt

# %%time
tmp['author_org_in_orgs_b_times'] = tmp.apply(lambda row: func(row['author_org'], row['orgs_b']), axis=1)

def func(a, b):
    b = set([y for x in b for y in x])
    # b = set([x for x in b])
    a = set(a)
    return len(a & b)
print(82*'-')
# %%time
tmp['author_interset_num'] = tmp.apply(lambda row: func(row['authors_a'], row['authors_b']), axis=1)

tmp['author_interset_num/paper_ids_len'] = (tmp['author_interset_num'] / (data['paper_ids_len'] - (data['label'] == 1).astype(int))).fillna(0)

tmp.head()

tmp['author_interset_num'].value_counts()

tmp[['author_org_in_orgs_b_times', 'author_interset_num', 'author_interset_num/paper_ids_len']].to_pickle('./feat/tmp.pkl')

(3142238, 5)
(505765, 4)
(3648003, 21)



Index(['author_id', 'author_name', 'author_org', 'label', 'paper_id',
       'abstract_a', 'authors_a', 'keywords_a', 'title_a', 'venue_a', 'year_a',
       'orgs_a', 'paper_ids', 'paper_ids_len', 'abstract_b', 'keywords_b',
       'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b', 'idx'],
      dtype='object')
----------------------------------------------------------------------------------
CPU times: user 16min 32s, sys: 1min 16s, total: 17min 48s
Wall time: 16min 57s


In [2]:
# !pip install catboost
import os
print("=================================切换目录=====================================")
print(os.getcwd()) # 打印当前工作目录
os.chdir('/home/shichaoyu/perl5/IRpro2/dataset/')
print(os.getcwd())
print(os.listdir())

## 7 baseline_v1 模型训练预测，结果生成
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import numpy as np
import gc
import time

out_dir = './out/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

from sklearn.preprocessing import LabelEncoder


from catboost import CatBoostClassifier, Pool

train_data = pd.read_pickle('./pkl2/train_data.pkl')
test_data = pd.read_pickle('./pkl2/valid_data.pkl')

data = pd.concat([train_data, test_data]).reset_index(drop=True)

data.head()

print(train_data.shape)
print(test_data.shape)
print(data.shape)

##
time_feat_a = pd.read_pickle('./feat/time_feat_a.pkl')
tmp = pd.read_pickle('./feat/tmp.pkl')

feats = [time_feat_a, tmp]

data = pd.concat([data] + feats, axis=1)

data.head()

drop_feat = ['author_id', 'author_name', 'author_org', 'paper_id', 'label']
used_feat = [c for c in data.columns if c not in drop_feat]
print(len(used_feat))
print(used_feat)

train = data[:len(train_data)]
test = data[len(train_data):]

test_x = test[used_feat]

# cv split according to author names
train_author_name = train['author_name'].unique()
print(len(train_author_name))

def gen_dict(df, label):
    df = df[['paper_id', 'author_name', 'author_id', label]]
    res = df.groupby(['paper_id', 'author_name'])[label].apply(np.argmax).reset_index()
    res.columns = ['paper_id', 'author_name', 'index']
    idx_name = df[['author_id']].reset_index()
    res = res.merge(idx_name, 'left', 'index')
    from collections import defaultdict
    res_dict = defaultdict(list)
    for pid, aid in res[['paper_id', 'author_id']].values:
        res_dict[aid].append(pid)
    return res_dict

def f1_score(pred_dict, true_dict):
    total_unassigned_paper = np.sum([len(l) for l in true_dict.values()])
    print('total_unassigned_paper: ', total_unassigned_paper)
    print('true author num: ', len(true_dict))
    author_weight = dict((k, len(v) / total_unassigned_paper) for k, v in true_dict.items())
    author_precision = {}
    author_recall = {}
    for author in author_weight.keys():
        # total pred, total belong, correct pred
        total_belong = len(true_dict[author])
        total_pred = (len(pred_dict[author]) if author in pred_dict else 0)
        correct_pred = len(set(true_dict[author]) & (set(pred_dict[author]) if author in pred_dict else set()))
        author_precision[author] = (correct_pred/total_pred) if total_pred > 0 else 0
        author_recall[author] = correct_pred / total_belong
        
    weighted_precision = 0
    weighted_recall = 0
    for author, weight in author_weight.items():
        weighted_precision += weight * author_precision[author]
        weighted_recall += weight * author_recall[author]
    weighted_f1 = 2 * weighted_precision * weighted_recall / (weighted_precision + weighted_recall)
    print('weighted_precision: %f, weighted_recall: %f, weighted_f1: %f' %(weighted_precision, weighted_recall, weighted_f1))
    return weighted_precision, weighted_recall, weighted_f1



=================================切换目录=====================================
/home/shichaoyu/perl5/IRpro2
/home/shichaoyu/perl5/IRpro2/dataset
['train', 'cat_feature_index.3c4c9673-11b0f6ac-cb85fb96-91833486.tmp', 'catboost_info', 'cat_feature_index.40b30d8b-f30e5088-cc5f7d50-21dddf23.tmp', 'out', 'cna_test_data', 'cna_data', 'pkl2', 'cat_feature_index.45ed2826-847e7041-c540e66c-339d9b99.tmp', 'cat_feature_index.3be0c692-8a8fe0c8-bb8954c3-56fb0ff.tmp', 'feat']
(3142238, 5)
(505765, 4)
(3648003, 5)
15
['year_a', 'year_b_min', 'year_b_max', 'year_b_mean', 'year_b_std', 'year_b_mm2', 'year_b_min-year_a', 'year_b_max-year_a', 'year_b_mean-year_a', 'year_b_mm2-year_a', 'year_inside_range', 'year_b_ai2', 'author_org_in_orgs_b_times', 'author_interset_num', 'author_interset_num/paper_ids_len']
220


###  提交日志

#### n=10
1. **0.619934746675664**  fold 10 round 1464 : auc: 0.982882 | mean auc 0.985608 | F1: 0.944590 | mean F1: 0.947347   n=10
      iterations=10000,  learning_rate=0.08, early_stopping_rounds=300,loss_function='CrossEntropy', 
      
2. **0.620929218125254** fold 10 round 1594 : auc: 0.982975 | mean auc 0.985650 | F1: 0.945342 | mean F1: 0.948062  
      iterations=10000,  learning_rate=0.08, **early_stopping_rounds=400**,loss_function='CrossEntropy',   n=10
      
3. **0.621775397186683**  fold 10 round 1341 : auc: 0.983031 | mean auc 0.985643 | F1: 0.943900 | mean F1: 0.947486
      **iterations=15000**,  learning_rate=0.08, early_stopping_rounds=300,loss_function='CrossEntropy',   n=10
     
4. **0.621081510328536**  fold 10 round 1359 : auc: 0.982904 | mean auc 0.985647 | F1: 0.944841 | mean F1: 0.947348
      **iterations=16000**,  learning_rate=0.08, early_stopping_rounds=300,loss_function='CrossEntropy',   n=10   

5. **0.622136901630442**  fold 10 round 998 : auc: 0.982874 | mean auc 0.985600 | F1: 0.944098 | mean F1: 0.947723 
      iterations=10000,  learning_rate=0.08,**early_stopping_rounds=500**,loss_function='CrossEntropy',   n=10 

6. **0.621305163601349** fold 10 round 983 : auc: 0.982874 | mean auc 0.985610 | F1: 0.945187 | mean F1: 0.948160 
      iterations=10000,  learning_rate=0.08,**early_stopping_rounds=550**,loss_function='CrossEntropy',   n=10 
      
 ===============================================================================================
 
6. **0.620987876156855** 
      iterations=12000,  learning_rate=0.08,**early_stopping_rounds=300**,loss_function='CrossEntropy',   n=10 
      
#### n=5

1. **0.620902879499788**  fold 10 round 816 : auc: 0.982305 | mean auc 0.985375 | F1: 0.956018 | mean F1: 0.959864
      iterations=10000,  learning_rate=0.08, early_stopping_rounds=300,loss_function='CrossEntropy',   n=5

2. **0.620202428314349**  fold 10 round 1048 : auc: 0.982160 | mean auc 0.985361 | F1: 0.955270 | mean F1: 0.960090
      iterations=15000,  learning_rate=0.08, early_stopping_rounds=300,loss_function='CrossEntropy',   n=5
      

In [7]:
# 8. 参数调整
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='3,4,5'

from sklearn.model_selection import KFold
preds = np.zeros((test.shape[0], 2))
scores = []
f1_scores = []
has_saved = False
imp = pd.DataFrame()
imp['feat'] = used_feat

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_author_name)):
    print('*' * 30)
#     验证集，测试集索引  同时得到对应的数据集，进行交叉验证
    trn_aname, val_aname = train_author_name[tr_idx], train_author_name[va_idx]
    trn_dat = train[train['author_name'].isin(trn_aname)]
    val_dat = train[train['author_name'].isin(val_aname)]
    print('*' * 30)
    X_train, y_train, X_valid, y_valid = trn_dat[used_feat], trn_dat['label'], val_dat[used_feat], val_dat['label']
    cate_features = []
    train_pool = Pool(X_train, y_train, cat_features=cate_features)
    eval_pool = Pool(X_valid, y_valid,cat_features=cate_features)
    print('*' * 30)
    if not has_saved: 
        cbt_model = CatBoostClassifier(iterations=21000,
                           learning_rate=0.08,
                           eval_metric='AUC',
                           use_best_model=True,
#                            leaf_estimation_method='Newton',
                           random_seed=42,
                           logging_level='Verbose',
                           task_type='GPU',
                           devices='5',
                           gpu_ram_part=0.5,
                           early_stopping_rounds=300,
                           loss_function='Logloss',
#                             depth=8,
                           )
        cbt_model.fit(train_pool, eval_set=eval_pool, verbose=100)
#         with open('./models/fold%d_cbt_v1.mdl' % index, 'wb') as file:
#             pickle.dump(cbt_model, file)
#     else:
#         with open('./models/fold%d_cbt_v1.mdl' % index, 'rb') as file:
#             cbt_model = pickle.load(file)
    
    imp['score%d' % (index+1)] = cbt_model.feature_importances_
    
    val_dat['pred'] = cbt_model.predict_proba(X_valid)[:, 1]
    val_pred_dict = gen_dict(val_dat, 'pred')
    val_true_dict = gen_dict(val_dat, 'label')
    precision, recall, f1 = f1_score(val_pred_dict, val_true_dict)
    f1_scores.append(f1)
    
    score = cbt_model.best_score_['validation']['AUC']
    scores.append(score)
    print('fold %d round %d : auc: %.6f | mean auc %.6f | F1: %.6f | mean F1: %.6f' % (index+1, cbt_model.best_iteration_, score,np.mean(scores), f1, np.mean(f1_scores))) 
    preds += cbt_model.predict_proba(test_x)  
#     break
    del cbt_model, train_pool, eval_pool
    del X_train, y_train, X_valid, y_valid
    import gc
    gc.collect()
    
#     mdls.append(cbt_model)

imp.sort_values(by='score1', ascending=False)

test_data['pred'] = preds[:, 1]

test_data.head()

result_dict = gen_dict(test_data, 'pred')

len(result_dict)

import json
import time
localtime = time.localtime(time.time())
save_path = './out/result_%02d%02d%02d%02d%02d.json' % (localtime[1], localtime[2], localtime[3], localtime[4],localtime[5])
with open(save_path, 'w') as file:
    file.write(json.dumps(result_dict))

******************************
******************************
******************************


CatBoostError: catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [14]:
# 8. 参数调整
from sklearn.model_selection import KFold
preds = np.zeros((test.shape[0], 2))
scores = []
f1_scores = []
has_saved = False
imp = pd.DataFrame()
imp['feat'] = used_feat

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_author_name)):
    print('*' * 30)
#     验证集，测试集索引  同时得到对应的数据集，进行交叉验证
    trn_aname, val_aname = train_author_name[tr_idx], train_author_name[va_idx]
    trn_dat = train[train['author_name'].isin(trn_aname)]
    val_dat = train[train['author_name'].isin(val_aname)]
    print('*' * 30)
    X_train, y_train, X_valid, y_valid = trn_dat[used_feat], trn_dat['label'], val_dat[used_feat], val_dat['label']
    cate_features = []
    train_pool = Pool(X_train, y_train, cat_features=cate_features)
    eval_pool = Pool(X_valid, y_valid,cat_features=cate_features)
    print('*' * 30)
    if not has_saved: 
        cbt_model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.08,
                           eval_metric='AUC',
                           use_best_model=True,
#                            leaf_estimation_method='Newton',
                           random_seed=42,
                           logging_level='Verbose',
                           task_type='GPU',
                           devices=[1,2],
                           gpu_ram_part=0.5,
                           early_stopping_rounds=550,
                           loss_function='CrossEntropy',
#                             depth=8,
                           )
        cbt_model.fit(train_pool, eval_set=eval_pool, verbose=100)
#         with open('./models/fold%d_cbt_v1.mdl' % index, 'wb') as file:
#             pickle.dump(cbt_model, file)
#     else:
#         with open('./models/fold%d_cbt_v1.mdl' % index, 'rb') as file:
#             cbt_model = pickle.load(file)
    
    imp['score%d' % (index+1)] = cbt_model.feature_importances_
    
    val_dat['pred'] = cbt_model.predict_proba(X_valid)[:, 1]
    val_pred_dict = gen_dict(val_dat, 'pred')
    val_true_dict = gen_dict(val_dat, 'label')
    precision, recall, f1 = f1_score(val_pred_dict, val_true_dict)
    f1_scores.append(f1)
    
    score = cbt_model.best_score_['validation']['AUC']
    scores.append(score)
    print('fold %d round %d : auc: %.6f | mean auc %.6f | F1: %.6f | mean F1: %.6f' % (index+1, cbt_model.best_iteration_, score,np.mean(scores), f1, np.mean(f1_scores))) 
    preds += cbt_model.predict_proba(test_x)  
#     break
    del cbt_model, train_pool, eval_pool
    del X_train, y_train, X_valid, y_valid
    import gc
    gc.collect()
    
#     mdls.append(cbt_model)

imp.sort_values(by='score1', ascending=False)

test_data['pred'] = preds[:, 1]

test_data.head()

result_dict = gen_dict(test_data, 'pred')

len(result_dict)

import json
import time
localtime = time.localtime(time.time())
save_path = './out/result_%02d%02d%02d%02d%02d.json' % (localtime[1], localtime[2], localtime[3], localtime[4],localtime[5])
with open(save_path, 'w') as file:
    file.write(json.dumps(result_dict))

******************************
******************************
******************************


0:	learn: 0.9741388	test: 0.9733723	best: 0.9733723 (0)	total: 127ms	remaining: 2m 6s
100:	learn: 0.9847890	test: 0.9841924	best: 0.9841927 (99)	total: 8.13s	remaining: 1m 12s
200:	learn: 0.9858270	test: 0.9850225	best: 0.9850236 (199)	total: 16.4s	remaining: 1m 5s
300:	learn: 0.9863668	test: 0.9852910	best: 0.9852929 (299)	total: 24.8s	remaining: 57.6s
400:	learn: 0.9867468	test: 0.9854158	best: 0.9854159 (399)	total: 33.3s	remaining: 49.8s
500:	learn: 0.9870386	test: 0.9854746	best: 0.9854746 (500)	total: 41.6s	remaining: 41.5s
600:	learn: 0.9872616	test: 0.9855222	best: 0.9855261 (596)	total: 49.8s	remaining: 33.1s
700:	learn: 0.9874696	test: 0.9854851	best: 0.9855261 (596)	total: 58.1s	remaining: 24.8s
800:	learn: 0.9876445	test: 0.9855247	best: 0.9855261 (596)	total: 1m 6s	remaining: 16.5s
900:	learn: 0.9878028	test: 0.9855166	best: 0.9855345 (822)	total: 1m 14s	remaining: 8.16s
999:	learn: 0.9879828	test: 0.9855111	best: 0.9855509 (930)	total: 1m 22s	remaining: 0us
bestTest = 0.9

0:	learn: 0.9747756	test: 0.9708628	best: 0.9708628 (0)	total: 20.7ms	remaining: 20.7s
100:	learn: 0.9847887	test: 0.9829091	best: 0.9829091 (100)	total: 7.99s	remaining: 1m 11s
200:	learn: 0.9858344	test: 0.9837517	best: 0.9837517 (200)	total: 16.2s	remaining: 1m 4s
300:	learn: 0.9864483	test: 0.9841740	best: 0.9841811 (299)	total: 24.1s	remaining: 55.9s
400:	learn: 0.9867832	test: 0.9844134	best: 0.9844134 (400)	total: 32s	remaining: 47.8s
500:	learn: 0.9870788	test: 0.9846603	best: 0.9846805 (489)	total: 40s	remaining: 39.9s
600:	learn: 0.9873474	test: 0.9848300	best: 0.9848300 (600)	total: 48.2s	remaining: 32s
700:	learn: 0.9875251	test: 0.9848728	best: 0.9848739 (698)	total: 56.4s	remaining: 24.1s
800:	learn: 0.9876844	test: 0.9850156	best: 0.9850158 (798)	total: 1m 4s	remaining: 16.1s
900:	learn: 0.9878623	test: 0.9850357	best: 0.9850537 (891)	total: 1m 13s	remaining: 8.06s
999:	learn: 0.9880017	test: 0.9849695	best: 0.9850537 (891)	total: 1m 21s	remaining: 0us
bestTest = 0.98505

0:	learn: 0.9740846	test: 0.9740320	best: 0.9740320 (0)	total: 222ms	remaining: 3m 41s
100:	learn: 0.9849703	test: 0.9839569	best: 0.9839569 (100)	total: 8.53s	remaining: 1m 15s
200:	learn: 0.9860473	test: 0.9846439	best: 0.9846458 (199)	total: 16.8s	remaining: 1m 6s
300:	learn: 0.9865253	test: 0.9848561	best: 0.9848569 (299)	total: 24.9s	remaining: 57.8s
400:	learn: 0.9868927	test: 0.9849120	best: 0.9849443 (368)	total: 33.2s	remaining: 49.5s
500:	learn: 0.9871950	test: 0.9849870	best: 0.9849870 (500)	total: 41.6s	remaining: 41.5s
600:	learn: 0.9874961	test: 0.9850749	best: 0.9850749 (600)	total: 49.8s	remaining: 33.1s
700:	learn: 0.9876885	test: 0.9850677	best: 0.9850819 (610)	total: 58.3s	remaining: 24.9s
800:	learn: 0.9878553	test: 0.9851208	best: 0.9851221 (799)	total: 1m 6s	remaining: 16.6s
900:	learn: 0.9880077	test: 0.9851296	best: 0.9851393 (885)	total: 1m 14s	remaining: 8.23s
999:	learn: 0.9881728	test: 0.9851024	best: 0.9851393 (885)	total: 1m 22s	remaining: 0us
bestTest = 0

0:	learn: 0.9732037	test: 0.9719571	best: 0.9719571 (0)	total: 155ms	remaining: 2m 35s
100:	learn: 0.9846037	test: 0.9846744	best: 0.9846744 (100)	total: 7.91s	remaining: 1m 10s
200:	learn: 0.9856971	test: 0.9854196	best: 0.9854196 (200)	total: 15.8s	remaining: 1m 2s
300:	learn: 0.9862596	test: 0.9857939	best: 0.9857939 (300)	total: 24.1s	remaining: 56s
400:	learn: 0.9867012	test: 0.9859315	best: 0.9859357 (394)	total: 32.3s	remaining: 48.2s
500:	learn: 0.9870234	test: 0.9860500	best: 0.9860541 (497)	total: 40.4s	remaining: 40.3s
600:	learn: 0.9872608	test: 0.9861086	best: 0.9861246 (565)	total: 48.3s	remaining: 32.1s
700:	learn: 0.9874603	test: 0.9861423	best: 0.9861441 (691)	total: 56.7s	remaining: 24.2s
800:	learn: 0.9876494	test: 0.9862029	best: 0.9862089 (792)	total: 1m 4s	remaining: 16.1s
900:	learn: 0.9877899	test: 0.9862325	best: 0.9862334 (899)	total: 1m 13s	remaining: 8.09s
999:	learn: 0.9879458	test: 0.9862536	best: 0.9862556 (993)	total: 1m 22s	remaining: 0us
bestTest = 0.9

...

0:	learn: 0.9741390	test: 0.9717250	best: 0.9717250 (0)	total: 139ms	remaining: 2m 18s
100:	learn: 0.9850613	test: 0.9821891	best: 0.9821891 (100)	total: 8.27s	remaining: 1m 13s
200:	learn: 0.9860506	test: 0.9831073	best: 0.9831170 (195)	total: 16.4s	remaining: 1m 5s
300:	learn: 0.9865788	test: 0.9834190	best: 0.9834218 (293)	total: 24.7s	remaining: 57.4s
400:	learn: 0.9870408	test: 0.9836527	best: 0.9836527 (400)	total: 33s	remaining: 49.3s
500:	learn: 0.9873446	test: 0.9836993	best: 0.9837013 (481)	total: 40.9s	remaining: 40.7s
600:	learn: 0.9875663	test: 0.9836786	best: 0.9837096 (512)	total: 49s	remaining: 32.5s
700:	learn: 0.9878179	test: 0.9837478	best: 0.9837685 (688)	total: 57.6s	remaining: 24.6s
800:	learn: 0.9879855	test: 0.9837441	best: 0.9837685 (688)	total: 1m 6s	remaining: 16.4s
900:	learn: 0.9881680	test: 0.9837105	best: 0.9837685 (688)	total: 1m 14s	remaining: 8.22s
999:	learn: 0.9883212	test: 0.9837069	best: 0.9837685 (688)	total: 1m 23s	remaining: 0us
bestTest = 0.983

0:	learn: 0.9741672	test: 0.9732900	best: 0.9732900 (0)	total: 64.3ms	remaining: 1m 4s
100:	learn: 0.9848627	test: 0.9846819	best: 0.9846819 (100)	total: 7.94s	remaining: 1m 10s
200:	learn: 0.9859210	test: 0.9853123	best: 0.9853123 (200)	total: 16.4s	remaining: 1m 5s
300:	learn: 0.9863819	test: 0.9855202	best: 0.9855202 (300)	total: 24.8s	remaining: 57.6s
400:	learn: 0.9867551	test: 0.9856734	best: 0.9856739 (398)	total: 33.1s	remaining: 49.5s
500:	learn: 0.9870594	test: 0.9857694	best: 0.9857694 (500)	total: 41.1s	remaining: 40.9s
600:	learn: 0.9873326	test: 0.9858515	best: 0.9858519 (598)	total: 49.3s	remaining: 32.7s
700:	learn: 0.9875487	test: 0.9858585	best: 0.9858631 (662)	total: 57.5s	remaining: 24.5s
800:	learn: 0.9877342	test: 0.9858438	best: 0.9858631 (662)	total: 1m 5s	remaining: 16.4s
900:	learn: 0.9878948	test: 0.9858500	best: 0.9858631 (662)	total: 1m 13s	remaining: 8.13s
999:	learn: 0.9880409	test: 0.9858836	best: 0.9858858 (996)	total: 1m 21s	remaining: 0us
bestTest = 0

0:	learn: 0.9736023	test: 0.9771281	best: 0.9771281 (0)	total: 156ms	remaining: 2m 36s
100:	learn: 0.9845552	test: 0.9859419	best: 0.9859566 (99)	total: 8.18s	remaining: 1m 12s
200:	learn: 0.9856108	test: 0.9867259	best: 0.9867288 (195)	total: 15.8s	remaining: 1m 2s
300:	learn: 0.9861561	test: 0.9869714	best: 0.9869768 (298)	total: 24.2s	remaining: 56.3s
400:	learn: 0.9865126	test: 0.9871359	best: 0.9871359 (400)	total: 32s	remaining: 47.9s
500:	learn: 0.9868208	test: 0.9872388	best: 0.9872441 (490)	total: 40.2s	remaining: 40.1s
600:	learn: 0.9870730	test: 0.9873314	best: 0.9873491 (589)	total: 48.4s	remaining: 32.1s
700:	learn: 0.9872912	test: 0.9873912	best: 0.9873988 (692)	total: 56.6s	remaining: 24.1s
800:	learn: 0.9874997	test: 0.9873504	best: 0.9873988 (692)	total: 1m 5s	remaining: 16.2s
900:	learn: 0.9876644	test: 0.9873019	best: 0.9873988 (692)	total: 1m 13s	remaining: 8.08s
999:	learn: 0.9878344	test: 0.9873140	best: 0.9873988 (692)	total: 1m 21s	remaining: 0us
bestTest = 0.98

0:	learn: 0.9743982	test: 0.9753079	best: 0.9753079 (0)	total: 84.1ms	remaining: 1m 24s
100:	learn: 0.9848181	test: 0.9849758	best: 0.9849758 (100)	total: 8.47s	remaining: 1m 15s
200:	learn: 0.9857537	test: 0.9853680	best: 0.9853697 (196)	total: 16.2s	remaining: 1m 4s
300:	learn: 0.9863255	test: 0.9856862	best: 0.9856902 (299)	total: 24.4s	remaining: 56.6s
400:	learn: 0.9867280	test: 0.9858955	best: 0.9858961 (399)	total: 32.8s	remaining: 48.9s
500:	learn: 0.9870069	test: 0.9859556	best: 0.9859575 (499)	total: 41.2s	remaining: 41s
600:	learn: 0.9872673	test: 0.9859295	best: 0.9859751 (514)	total: 49.6s	remaining: 32.9s
700:	learn: 0.9874999	test: 0.9859340	best: 0.9859751 (514)	total: 58s	remaining: 24.7s
800:	learn: 0.9876391	test: 0.9859482	best: 0.9859751 (514)	total: 1m 6s	remaining: 16.5s
900:	learn: 0.9878050	test: 0.9858979	best: 0.9859751 (514)	total: 1m 14s	remaining: 8.22s
999:	learn: 0.9879754	test: 0.9859595	best: 0.9859751 (514)	total: 1m 22s	remaining: 0us
bestTest = 0.98

0:	learn: 0.9745938	test: 0.9692506	best: 0.9692506 (0)	total: 108ms	remaining: 1m 47s
100:	learn: 0.9850067	test: 0.9809700	best: 0.9809700 (100)	total: 8.19s	remaining: 1m 12s
200:	learn: 0.9860573	test: 0.9819126	best: 0.9819126 (200)	total: 16.7s	remaining: 1m 6s
300:	learn: 0.9865771	test: 0.9822334	best: 0.9822427 (293)	total: 25.3s	remaining: 58.7s
400:	learn: 0.9869813	test: 0.9824619	best: 0.9824634 (395)	total: 33.6s	remaining: 50.2s
500:	learn: 0.9872648	test: 0.9826027	best: 0.9826184 (486)	total: 42s	remaining: 41.8s
600:	learn: 0.9874830	test: 0.9826946	best: 0.9827181 (589)	total: 49.9s	remaining: 33.1s
700:	learn: 0.9876997	test: 0.9827538	best: 0.9827698 (686)	total: 58.1s	remaining: 24.8s
800:	learn: 0.9878742	test: 0.9828046	best: 0.9828047 (790)	total: 1m 6s	remaining: 16.6s
900:	learn: 0.9880486	test: 0.9828023	best: 0.9828233 (862)	total: 1m 14s	remaining: 8.23s
999:	learn: 0.9881766	test: 0.9828618	best: 0.9828739 (983)	total: 1m 23s	remaining: 0us
bestTest = 0.9

In [15]:
%whos

Variable                       Type             Data/Info
---------------------------------------------------------
CatBoostClassifier             type             <class 'catboost.core.CatBoostClassifier'>
KFold                          ABCMeta          <class 'sklearn.model_selection._split.KFold'>
LabelEncoder                   type             <class 'sklearn.preproces<...>sing.label.LabelEncoder'>
OrderedDict                    type             <class 'collections.OrderedDict'>
Pool                           type             <class 'catboost.core.Pool'>
aid                            str_             Ms4yXx88
an                             str              Jian-Qin Sun
ao                             OrderedDict      OrderedDict([('name', 'Ji<...>partment of Nutrition')])
author_id                      str              3qn5CxG9
author_ids                     list             n=11
author_ids_sample              set              {'Izy4g5GJ', 'vwt9Wd81', <...>, 'xOEKHNxv', 'Ms4yXx88'}